# Seattle AirBNB Data Analysis

## Importing the necessary libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
import seaborn as sns


%matplotlib inline

#pd.set_option('display.float_format', lambda x: '%.0f' % x)

### Importing the Dataset

In [ ]:
#listings = pd.read_csv("listings.csv", parse_dates=["host_since", "first_review", "last_review"])
listings = pd.read_csv("listings.csv")

## Exploring the Dataset

In [ ]:
listings.head(2)

In [ ]:
# Continous Variables
listings.describe()

In [ ]:
listings.hist();

In [ ]:
#sns.heatmap(listings.corr(), annot=True, fmt='.2f');

In [ ]:
X = listings[[]]
y = listings['price']

In [ ]:
# Show the column types
listings.info()

## Basic Data Cleaning

Let us check for unique values in every column

In [ ]:
# 5.3 Identify Columns that Contain a Single Value
# Get number of unique values in each column
listings.nunique().sort_values()

In [ ]:
# 5.4 Delete Columns That Contain a Single Value
# Remove these because of zero-variance predictors
print(listings.shape)
# Get number of unique values in each column
counts = listings.nunique()
# record columns to delete
to_del = [i for i,v in enumerate(counts) if v == 1 ]
print(to_del)
# drop unless columns
listings.drop(listings.columns[to_del], axis=1, inplace=True)
print(listings.shape)

In [ ]:
# 5.5 Consider Columns that Have Very Few Values
print(listings.shape)
# get number of unique values for each column
counts = listings.nunique()
# record columns to delete
to_del = [i for i,v in enumerate(counts) if (float(v)/listings.shape[0]*100) < 1]
print(to_del)
# drop unless columns
listings.drop(listings.columns[to_del], axis=1, inplace=True)
print(listings.shape)

In [ ]:
most_missing_cols = set(listings.columns[listings.isnull().mean() > 0.75])
most_missing_cols

In [ ]:
# Drop columns with more than 75% of rows missing
print(listings.shape)
listings.drop(most_missing_cols, axis=1, inplace=True);
print(listings.shape);

In [ ]:
# 5.5 Summarize the percentage of unique values for each columnn using numpy
#from numpy import unique
#from numpy import loadtxt
# load the dataset
#data = loadtxt("listings.csv", delimiter=",")
# summarize the number of unique values in each column
# for i in range(listings.shape[1]):
#    num = len(unique(data[:, i]))
#    percentage = float(num) / listings.shape[0] * 100
#    if percentage < 1:
#        print('%d, %d, %.1f%%' % (i, num, percentage))

In [ ]:
# 5.6 Remove Columns That Have A Low Variance
#counts = listings.nunique().sort_values(ascending=False)
# record columns to delete
#to_del = [i for i,v in enumerate(counts) if (float(v)/listings.shape[0]*100) < 1]
#to_del

In [ ]:
# Remove the columns with urls
print(listings.shape)
# record columns to delete
to_del = listings.filter(like='_url').columns
print(to_del)
# drop unless columns
listings.drop(to_del, axis=1, inplace=True)
print(listings.shape)   

In [ ]:
# 5.7 Identify Rows That Contain Duplicate Data
# print(listings.shape)
# calculate duplicates
dups = listings.duplicated()
# report if there are duplicates
print(dups.any())
# list all duplicate rows
print(listings[dups])

In [ ]:
# 5.8 Delete Rows That Contain Duplicate Data
# print(listings.shape)
# calculate duplicates
dups = listings.duplicated()
# report if there are duplicates
print(dups.any())
# list all duplicate rows
print(listings[dups])
# delete duplicate rows
listings.drop_duplicates(inplace=True)
print(listings.shape)

In [ ]:
# The price columns are showing as objects or strings

# Remove $ and convert 'price' to float
#listings_price_cols = listings.filter(like='price').columns
listings["price"] = listings["price"].str.replace('$', '', regex=True)
listings["price"] = pd.to_numeric(listings["price"], errors='coerce');

listings["weekly_price"] = listings["weekly_price"].str.replace('$', '', regex=True)
listings["weekly_price"] = pd.to_numeric(listings["weekly_price"], errors='coerce');

listings["monthly_price"] = listings["monthly_price"].str.replace('$', '', regex=True)
listings["monthly_price"] = pd.to_numeric(listings["monthly_price"], errors='coerce');


#listings_deposit_fee_cols = listings[["security_deposit","cleaning_fee"]]
listings["security_deposit"] = listings["security_deposit"].str.replace('$', '', regex=True)
listings["security_deposit"] = pd.to_numeric(listings["security_deposit"], errors='coerce');

listings["cleaning_fee"] = listings["cleaning_fee"].str.replace('$', '', regex=True)
listings["cleaning_fee"] = pd.to_numeric(listings["cleaning_fee"], errors='coerce');

listings["extra_people"] = listings["extra_people"].str.replace('$', '', regex=True)
listings["extra_people"] = pd.to_numeric(listings["extra_people"], errors='coerce');

#listings.filter(like='price').columns

## Identify Row/Cols with Missing Data

In [ ]:
# List columns without nulls
set_cols_no_null = set(listings.columns[listings.isnull().sum() == 0])
set_cols_no_null

In [ ]:
# Columns without nulls
len(set_cols_no_null)

In [ ]:
# List columns with NaN values or nulls
list_cols_with_nulls = listings.columns[listings.isnull().sum() != 0].tolist()
list_cols_with_nulls

In [ ]:
# Columns with nulls
len(list_cols_with_nulls)

## Fill in Missing Data

In [ ]:
#get the list of all integer columns
int_cols = list(listings.select_dtypes('int64').columns)
print(f'Int Cols: , {int_cols}')
#fillna(df.dtypes.replace({'float64': 0.0, 'O': 'NULL'}), inplace=True)

#get the list of all float columns
float_cols = list(listings.select_dtypes('float64').columns)
print(f'Float Cols: , {float_cols}')

#get the list of all object columns
object_cols = list(listings.select_dtypes('object').columns)
print(f'object_cols: , {object_cols}')

#get the list of all datetime columns
datetime_cols = list(listings.select_dtypes('datetime64').columns)
print(f'Datetime64 Cols: , {datetime_cols}')

#looping through if each column to fillna
for i in int_cols:
    listings[i].fillna(0,inplace=True)

for f in float_cols:
    listings[f].fillna(0,inplace=True)

for o in object_cols:
    listings[o].fillna("NULL",inplace=True)
    
#for t in datetime_cols:
#    listings[t].set_index('timestamp').resample('S').ffill().reset_index()

In [ ]:
#get the list of all datetime columns
datetime_cols = list(listings.select_dtypes('datetime64').columns)
print(f'Datetime64 Cols: , {datetime_cols}')

for t in datetime_cols:
    listings[t].ffill(inplace=True)
    #print(listings[t])


In [ ]:
listings.hist(figsize=(20,8), grid=False, layout=(3,8));

In [ ]:
plt.figure(figsize = (20,5))
sns.heatmap(listings.corr(), annot=True, fmt='.2f', linewidths=.5);
#listings.corr()

In [ ]:
X = listings[['cleaning_fee', 'security_deposit', 'extra_people', 'maximum_nights']]
y = listings['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=42)

# Four Steps
# instantiate your model
# fit model to training data
# predict on test data
# score your model on the test

lm_model = LinearRegression(normalize=True)


In [ ]:
# fit model to training data
lm_model.fit(X_train, y_train)

In [ ]:
# predict on test data
y_pred = lm_model.predict(X_test)

In [ ]:
# score your model on the test
print(r2_score(y_test, y_pred))

## Encoding the Categorical Data

In [ ]:
obj_cols = listings.select_dtypes(include=['object'])
#obj_cols.cat.codes
obj_cols.columns
obj_cols.head(2)
#enc = OneHotEncoder(handle_unknown='ignore')
#for cols in obj_cols:
#    print(obj_cols)

## Visualize the Data

In [ ]:
host_neighbourhood_vals = listings.host_neighbourhood.value_counts()
(host_neighbourhood_vals[:10]/listings.shape[0]).plot(kind='bar')
plt.title("Top 10 Host Neighborhooods by Percentage of Listings");

In [ ]:
neighbourhood_vals = listings.neighbourhood.value_counts()
(neighbourhood_vals[:10]/listings.shape[0]).plot(kind='bar')
plt.title("Top 10 Neighborhooods by Listing Count");

In [ ]:
# Get Average Price by neighbourhood
listings.groupby(["neighbourhood"])['price'].mean().nlargest(10).plot(kind='bar')
plt.title("Top 10 Neighborhood by Average Price");

In [ ]:
# Get Average Price by neighbourhood
listings.groupby(["neighbourhood"])['price'].mean().nsmallest(10).plot(kind='bar')
plt.title("10 Least Expensive Rental Neighborhoods by Average Price");

In [ ]:
#listings.host_neighbourhood.value_counts()
print(listings.neighbourhood.value_counts().nlargest(10))
# List the top 10 neighborhoods by listing
listings.groupby(["neighbourhood"])['price'].count().nlargest(10).plot(kind='bar')
plt.title("Listing Count by Neighborhood");

In [ ]:
#listings.host_neighbourhood.value_counts()
print(listings.neighbourhood.value_counts().nlargest(10))
# List the top 10 neighborhoods by listing
listings.groupby(["neighbourhood"])['price'].count().nlargest(10).plot(kind='bar')
plt.title("Listing Count by Neighborhood");

## Statistics

In [ ]:
correlations = listings.corr()
# plot correlation matrix
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(correlations, vmin=-1, vmax=1)
fig.colorbar(cax)
plt.show()

In [ ]:
listings.corr()

In [ ]:
listings.info()

In [ ]:
# Continous Variables
listings.describe()

In [ ]:
# Scatterplot Matrix
#from matplotlib import pyplot as plt
#pd.plotting.scatter_matrix(listings[["neighbourhood","price"]])
#plt.xticks(rotation=45)
#plt.show()

#fig, ax = plt.subplots(figsize=(16,8))
#ax.scatter(listings['neighbourhood'], listings['price'])
#plt.xticks(rotation=45)
#plt.show()

#float64_df.plot.scatter(x='review_scores_rating', y='price', c='DarkBlue')
#listings.plot.scatter(x='neighbourhood', y='price', c='DarkBlue')
#float64_df.head(2)

In [ ]:
# pandas.plotting.scatter_matrix
pd.plotting.scatter_matrix(listings, alpha=0.2);

In [ ]:
#
listings.plot.scatter(x='neighbourhood', y='price', c='DarkBlue');

In [ ]:
# BoxPlot
listings.boxplot();